In [18]:
%matplotlib inline
import matplotlib.pyplot as plt # Module for plotting
import numpy as np
from scipy.signal import correlate
from scipy.integrate import odeint
from ipywidgets import interactive, FloatSlider

## Ex 1 - Orbital Mechanics

In [19]:
scale_factor = 39.48734
vi = np.sqrt(scale_factor)
r_mars = 1.52342
degtorad = np.pi/180
times = np.linspace(0., 0.75, 100)

def grav_derivs(state, t):
  x, y, vx, vy = state

  r =  np.sqrt(x**2+y**2)
  acc = scale_factor*r**-2
  acc_x = -acc*x/r
  acc_y = -acc*y/r
  return [vx, vy, acc_x, acc_y]

def launch(vel, angle):
  r_init = (1, 0, vel*np.sin(degtorad*angle), vi+vel*np.cos(degtorad*angle)) # FIXME: Set correct x and vy for Mars
  r_arr = odeint(grav_derivs, r_init, times)
  plt.figure(figsize=(8,8))
  ax = plt.gca()
  plt.scatter(0., 0., c='y', s=50)
  ax.add_patch(plt.Circle((0,0), 1, color='b', fill=False))
  ax.add_patch(plt.Circle((0,0), r_mars, color='r', fill=False))
  plt.scatter(r_arr[:,0], r_arr[:,1], c=times, s=8)
  plt.gca().set_aspect('equal', 'datalim')
  plt.show()

In [20]:
interactive_plot = interactive(launch,
                               vel=FloatSlider(0., min=0., max=2., step=0.02, description='Velocity'),
                               angle=FloatSlider(0., min=-180., max=180., step=5., description='Angle'))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='Velocity', max=2.0, step=0.02), FloatSlider(value=0.…

## Ex 2 - Nonlinear Equation

The van der Pol equation describes a nonlinearly damped oscillator. It can't be solved analytically, but using a computer it's no more difficult than the others. The equation is:

$$\frac{d^2 x}{dt^2} - \mu (1 - x^2) \frac{d x}{d t} + x = 0$$

**Exercise**: Make a phase plot of this for different $\mu$, as well as different initial conditions. Also plot $x(t)$. The [Wikipedia page](https://en.wikipedia.org/wiki/Van_der_Pol_oscillator) contains result you can compare against.

In [21]:
times = np.linspace(0, 10, 400)

def deriv(state, t, mu):
    x, vx = state
    dxdt = vx
    dvxdt = mu*(1-x**2)*vx - x
    return (dxdt, dvxdt) # Return a tuple of the derivatives.

def vanderpol(x0, v0, mu):
    x_init = (x0, v0)
    return odeint(deriv, (x0, v0), times, args=(mu,))

In [22]:
test_points = np.random.uniform(-4, 4, size=(40,2))

def plot(mu):
  plt.figure(figsize=(6,6))
  for x0, v0 in test_points:
    xarr  = vanderpol(x0, v0, mu)
    plt.scatter(x0, v0)
    plt.plot(xarr[:,0], xarr[:,1])
  plt.xlim(-6,6)
  plt.ylim(-8,8)
  plt.xlabel('Position')
  plt.ylabel('Velocity')
  plt.show()

interactive_plot = interactive(plot, mu=(0., 2.))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=1.0, description='mu', max=2.0), Output(layout=Layout(height='350px'))…

## Ex 3 - Matched Filtering

In [23]:
sample_rate = 32
num_data_samples = 32*sample_rate
times = np.arange(num_data_samples) / sample_rate

gaussian_width = 2
signal_inst_frequency = 2. + 1.0*np.sin(2 * np.pi * 0.1 * times)
phases = [0]
for i in range(1,len(times)):
    phases.append(phases[-1] + 2 * np.pi * signal_inst_frequency[i] * 1./sample_rate)

sinusoid = np.sin(phases)
gaussian = np.exp( - (times - 16)**2 / (2 * gaussian_width))
signal = sinusoid * gaussian

template = signal[11*sample_rate:21*sample_rate]

noise = np.random.normal(size=num_data_samples)

In [24]:
def matched_filter(amplitude):
  data = noise+amplitude*signal

  fig, axs = plt.subplots(1, 2, figsize=(16,6))
  axs[0].plot(times, noise+amplitude*signal)
  axs[0].set_ylim(-6,6)
  axs[1].plot(times, np.abs(correlate(data,template, mode='same')))
  axs[1].set_ylim(0,160)
  plt.show()
  
interactive_plot = interactive(matched_filter,
                               amplitude=FloatSlider(0., min=0., max=4., step=0.02, description='Signal amplitude'))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='Signal amplitude', max=4.0, step=0.02), Output(layou…